In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from os.path import join as oj
import analyze_utils
import matplotlib.pyplot as plt
import iprompt
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
tqdm.pandas()

In [ ]:
keys = ['model_cls', 'task_name', 'checkpoint', 'seed']
keys_extra = ['n_shots', 'num_learned_tokens']
keys_out = ['reciprocal_rank', 'prefixes', 'iprompt_preprefix_str']

save_dir = '/home/chansingh/mntv1/iprompt_revision_xmas/'

r = analyze_utils.load_results_and_cache_autoprompt_json(save_dir, save_file='r.pkl', one_row_only=True)
r = pd.read_pickle(oj(save_dir, 'r.pkl'))

In [10]:
# # subselect the data
r = r[r['n_shots'] == 5]
r = r[r['num_learned_tokens'] == 6]
r = r[~r['model_cls'] == 'suffix'] # original suffix had a reranking step, correct one is just called "suff"

# # add more keys
keys_derived = ['task_collection', 'top_prompt_correctness', 'bleu_top_prompt']
r['task_collection'] = r.task_name.apply(analyze_utils.task_collection)
r['top_prompt_correctness'] = r['reciprocal_rank'] == 1
r['gt_prompt'] = r['task_name'].apply(lambda x: iprompt.data.TASKS[x]['description'])
r['bleu_top_prompt'] = r.progress_apply(lambda x: sentence_bleu(x['gt_prompt'].split(), x['prefixes'].split()), axis=1) # maybe need to split before calling?

d = r[keys + keys_extra + keys_out + keys_derived]
d.to_pickle(oj(save_dir, 'd.pkl'))
d  = pd.read_pickle(oj(save_dir, 'd.pkl'))

  0%|          | 0/1210 [00:00<?, ?it/s]/home/chansingh/.embgam/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/chansingh/.embgam/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/chansingh/.embgam/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Co

# Metrics table

In [12]:
metrics = d.groupby(by=['model_cls', 'task_collection', 'checkpoint']).mean(numeric_only=True).reset_index()
# with pd.option_context('display.float_format', lambda x: '%.1e' % x):
with pd.option_context('display.float_format', lambda x: '%.2g' % x):
    for ckpt in metrics.checkpoint.unique():
        print(ckpt)
        m = (
            metrics[metrics.checkpoint == ckpt]
            .pivot_table(index='model_cls', columns='task_collection', values=['reciprocal_rank', 'top_prompt_correctness', 'bleu_top_prompt'])
            .transpose()
        )
        display(m)

EleutherAI/gpt-j-6B


model_cls                               autoprompt  iprompt  suff   suffix
                       task_collection                                    
bleu_top_prompt        ANLI                      0 8.6e-233     0 1.3e-232
                       DD                        0 6.6e-233   NaN 3.8e-233
                       Induction          2.2e-233 1.6e-232   NaN 2.4e-232
                       Math                      0 3.2e-232   NaN 3.2e-232
reciprocal_rank        ANLI                  0.074     0.41  0.14    1e-10
                       DD                  0.00036   0.0091   NaN    0.088
                       Induction             0.086     0.35   NaN    0.071
                       Math                   0.14     0.69   NaN    0.083
top_prompt_correctness ANLI                  0.074     0.37     0        0
                       DD                        0        0   NaN    0.059
                       Induction              0.08     0.28   NaN    0.071
                       Math                   0.13      0.6   NaN    0.075

facebook/galactica-6.7b


model_cls                               autoprompt  iprompt   suffix
                       task_collection                              
bleu_top_prompt        ANLI                      0 3.3e-233 7.8e-232
                       DD                        0 1.4e-232   2e-232
                       Induction          2.1e-232 5.8e-233        0
                       Math                      0 2.3e-232        0
reciprocal_rank        ANLI                 0.0043     0.45    1e-10
                       DD                    1e-10     0.19    0.025
                       Induction             0.056     0.16    1e-10
                       Math                   0.06     0.24     0.33
top_prompt_correctness ANLI                      0      0.4        0
                       DD                        0     0.11        0
                       Induction             0.053    0.095        0
                       Math                   0.04     0.19     0.33

google/flan-t5-xl


model_cls                               iprompt   suffix
                       task_collection                  
bleu_top_prompt        ANLI            8.4e-233 1.4e-232
                       DD              1.8e-232 2.7e-233
                       Induction       6.9e-233 6.4e-233
                       Math            4.1e-233 1.2e-232
reciprocal_rank        ANLI                0.26    1e-10
                       DD                  0.11    0.033
                       Induction           0.17    0.013
                       Math                0.15    1e-10
top_prompt_correctness ANLI                 0.2        0
                       DD                 0.091    0.022
                       Induction           0.11        0
                       Math                 0.1        0

# Prompts table

In [21]:
for ckpt in d.checkpoint.unique():
    print(ckpt)
    prompts = d[d.checkpoint == ckpt]
    print(prompts.shape[0])
    prompts = (
        prompts
        .sort_values(by=['task_collection', 'reciprocal_rank'], ascending=False)
        .groupby(by=['model_cls', 'task_name'])
        .first()
    )
    # with pd.option_context('display.float_format', lambda x: '%.1e' % x):
    with pd.option_context('display.max_rows', None):
        display(prompts[['prefixes']].pivot_table(index='task_name', columns='model_cls', values='prefixes', aggfunc=lambda x: ' '.join(x)))

EleutherAI/gpt-j-6B
778


model_cls,autoprompt,iprompt,suff,suffix
task_name,,,,
active_to_passive,(= 18 the the subst,Choose a pronoun for each sentence,NaN,NaN
add_two,>:Returns Adding togetherFont accomplish,Create a function named `sum,NaN,n>2 m=
antonyms,čthe bectheBut But,The noun to its opposite (,NaN,To maximize the average amount of
cause_and_effect,REG Kinect virginity developed mosquit The,What would each sentence be if,NaN,NaN
common_concept,???????? parted configuredthe ????????,Find a noun that includes all,NaN,NaN
d3_0,NaN,line contains this string? No,NaN,NaN
d3_1,NaN,"sentence contains ""yes"" or",NaN,NaN
d3_10,NaN,"message contains ""no"". No",NaN,gives all outputs for each model
d3_11,NaN,"string contains ""No"" or",NaN,number does not have an operator


google/flan-t5-xl
218


model_cls,iprompt,suffix
task_name,,
active_to_passive,Answer: The judge encouraged the,Answer: The athlete was recognized
add_two,prompt: find the sum of,NaN
antonyms,What is the opposite of what,NaN
cause_and_effect,Answer: Mary's.,Answer: The water levels in
common_concept,answer: sewing wrenches,Answer: raccoon
d3_0,NaN,no if it is
d3_1,NaN,Answer: yes else: no
d3_10,Answer: no.||,NaN
d3_11,NaN,Answer: no; # flood


facebook/galactica-6.7b
214


model_cls,autoprompt,iprompt,suffix
task_name,,,
active_to_passive,NaN,4-way Multiple Choice,NaN
add_two,ě addthe Hyper� addi,In order to add two or,Evaluate all six of your answers
antonyms,meet equilibration stiptertead asymmetry,What is the opposite of each,NaN
cause_and_effect,shaking Dthethethethe,Find clues as to why each,Descriptions and Examples of Correct
d3_1,NaN,NaN,contains 5 words beginning with
d3_13,NaN,NaN,prompt contains an answer choice that
d3_14,NaN,NaN,has both negative polarity as well
d3_15,NaN,NaN,question contains Yes else output No
d3_17,NaN,data includes at least one Sem,NaN
